# Code for calculating pitch control and PCV in sliding time windows of 1 and 5 minutes. The 1-minute time window is used to calculated the 2-minute time windows for the effect of key moments on the flow of momentum in another file. The five-minute time window is used for the effect of managerial decision-making on the flow of momentum, and for calculating the 10-minute time windows for the impact of the flow of momentum on managerial decision-making. 

In [ ]:
#five minute calculations
import os
from progressbar import ProgressBar
import pandas as pd
import pickle
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code/Pitch control per minute'
home_files = []
away_files = []
#Create a list of all 100 files used for pitch control
for file in os.listdir(path):
    if 'dfhome' in file:
        home_files.append(file)
    elif 'dfaway' in file:
        away_files.append(file)
home_files = sorted(home_files)
away_files = sorted(away_files)
#iterate over each file
for file in range(len(home_files)):
    print(home_files[file])
    print(away_files[file])
    #import home and away dataframes
    df_home = pd.read_excel(home_files[file])
    df_away = pd.read_excel(away_files[file])
    df_home = df_home.set_index("Frame_number")
    df_away = df_away.set_index("Frame_number")
    
    #only do the calculations when the ball is in play
    df_home_alive = df_home.loc[df_home['Ball_status'] == 'Alive']
    df_away_alive = df_away.loc[df_away['Ball_status'] == 'Alive']
    #initialize lists for pitch control and PCV for the home and away team
    Pitch_control_home5 = []
    Pitch_control_away5 = []
    Weighted_pitch_control_home5 = []
    Weighted_pitch_control_away5 = []
    Time = []
    half = []
    start_frame_first_half = df_home.iloc[0].name
    second_half_df = df_home.loc[df_home["Half"] == 2]
    start_frame_second_half = second_half_df.iloc[0].name
    second_half_df_alive = df_home_alive.loc[df_home_alive["Half"] == 2]
    second_half_idx = second_half_df_alive.iloc[0].name
    pbar = ProgressBar()
    
    #iterate over the length of the dataframe
    for i in pbar(range(len(df_home_alive))):
        half.append(df_home_alive.iloc[i]['Half'])
        if df_home_alive.iloc[i]['Half'] == 1:
            #if first five minutes of the first half: add the average of the minutes played
            if df_home_alive.iloc[i].name < start_frame_first_half + 60*25*5:
                Pitch_control_home5.append(df_home_alive.loc[df_home_alive.iloc[0].name: df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Pitch_control_away5.append(1-df_home_alive.loc[df_home_alive.iloc[0].name: df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Weighted_pitch_control_home5.append(df_home_alive.loc[df_home_alive.iloc[0].name: df_home_alive.iloc[i].name+1]['Weighted_PPCF'].mean())
                Weighted_pitch_control_away5.append(df_away_alive.loc[df_away_alive.iloc[0].name: df_away_alive.iloc[i].name+1]['Weighted_PPCF'].mean())

            #else only select the five-minute time windows        
            else:
                Pitch_control_home5.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25*5].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Pitch_control_away5.append(1-df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25*5].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Weighted_pitch_control_home5.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25*5].min(): df_home_alive.iloc[i].name+1]['Weighted_PPCF'].mean())
                Weighted_pitch_control_away5.append(df_away_alive.loc[df_away_alive.index[df_away_alive.index >= df_away_alive.iloc[i].name - 60*25*5].min(): df_away_alive.iloc[i].name+1]['Weighted_PPCF'].mean())

            Time.append(df_home_alive.iloc[i]['Time_in_game']/60)
        #second half calculations
        if df_home_alive.iloc[i]['Half'] == 2:
            #add the average values for the time windows
            Pitch_control_home5.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25*5].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
            Pitch_control_away5.append(1-df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25*5].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
            Weighted_pitch_control_home5.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25*5].min(): df_home_alive.iloc[i].name+1]['Weighted_PPCF'].mean())
            Weighted_pitch_control_away5.append(df_away_alive.loc[df_away_alive.index[df_away_alive.index >= df_away_alive.iloc[i].name - 60*25*5].min(): df_away_alive.iloc[i].name+1]['Weighted_PPCF'].mean())       
            Time.append(df_home_alive.iloc[i]['Time_in_game']/60 + 45)

    data = {'PC home previous 5 minutes': Pitch_control_home5,
            'PC away previous 5 minutes': Pitch_control_away5,
            'Weighted PC home previous 5 minutes': Weighted_pitch_control_home5,
            'Weighted PC away previous 5 minutes': Weighted_pitch_control_away5,
            'Half': half,
            'Time': Time}

    #create pickle file for each match in the data for pitch control and PCV
    df = pd.DataFrame(data)
    pickle.dump(df, open('pc_over_last_five_minutes' + str(home_files[file][7:14]) + '.p', 'wb'))  
    
        

In [ ]:
#one-minute sliding time wondows
#same as for the previous block of code, but then in time wondows of one minute
import os
from progressbar import ProgressBar
import pandas as pd
import pickle
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code/Pitch control per minute'
home_files = []
away_files = []
for file in os.listdir(path):
    if 'dfhome' in file:
        home_files.append(file)
    elif 'dfaway' in file:
        away_files.append(file)
home_files = sorted(home_files)
away_files = sorted(away_files)
for file in range(len(home_files)):
    print(home_files[file])
    print(away_files[file])
    df_home = pd.read_excel(home_files[file])
    df_away = pd.read_excel(away_files[file])
    df_home = df_home.set_index("Frame_number")
    df_away = df_away.set_index("Frame_number")
    df_home_alive = df_home.loc[df_home['Ball_status'] == 'Alive']
    df_away_alive = df_away.loc[df_away['Ball_status'] == 'Alive']
    Pitch_control_home = []
    Pitch_control_away = []
    Weighted_pitch_control_home = []
    Weighted_pitch_control_away = []
    Time = []
    half = []
    start_frame_first_half = df_home.iloc[0].name
    second_half_df = df_home.loc[df_home["Half"] == 2]
    start_frame_second_half = second_half_df.iloc[0].name
    second_half_df_alive = df_home_alive.loc[df_home_alive["Half"] == 2]
    second_half_idx = second_half_df_alive.iloc[0].name
    pbar = ProgressBar()
    for i in pbar(range(len(df_home_alive))):
        half.append(df_home_alive.iloc[i]['Half'])
        if df_home_alive.iloc[i]['Half'] == 1:
            if df_home_alive.iloc[i].name < start_frame_first_half + 60*25:
                Pitch_control_home.append(df_home_alive.loc[df_home_alive.iloc[0].name: df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Pitch_control_away.append(1-df_home_alive.loc[df_home_alive.iloc[0].name: df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Weighted_pitch_control_home.append(df_home_alive.loc[df_home_alive.iloc[0].name: df_home_alive.iloc[i].name+1]['Weighted_PPCF'].mean())
                Weighted_pitch_control_away.append(df_away_alive.loc[df_away_alive.iloc[0].name: df_away_alive.iloc[i].name+1]['Weighted_PPCF'].mean())
            else:    
                Pitch_control_home.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Pitch_control_away.append(1-df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
                Weighted_pitch_control_home.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25].min(): df_home_alive.iloc[i].name+1]['Weighted_PPCF'].mean())
                Weighted_pitch_control_away.append(df_away_alive.loc[df_away_alive.index[df_away_alive.index >= df_away_alive.iloc[i].name - 60*25].min(): df_away_alive.iloc[i].name+1]['Weighted_PPCF'].mean())

            Time.append(df_home_alive.iloc[i]['Time_in_game']/60)
        if df_home_alive.iloc[i]['Half'] == 2:
            Pitch_control_home.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
            Pitch_control_away.append(1-df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25].min(): df_home_alive.iloc[i].name+1]['PPCF'].mean())
            Weighted_pitch_control_home.append(df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[i].name - 60*25].min(): df_home_alive.iloc[i].name+1]['Weighted_PPCF'].mean())
            Weighted_pitch_control_away.append(df_away_alive.loc[df_away_alive.index[df_away_alive.index >= df_away_alive.iloc[i].name - 60*25].min(): df_away_alive.iloc[i].name+1]['Weighted_PPCF'].mean())       
            Time.append(df_home_alive.iloc[i]['Time_in_game']/60 + 45)

    data = {'PC home previous minute': Pitch_control_home,
            'PC away previous minute': Pitch_control_away,
            'Weighted PC home previous minute': Weighted_pitch_control_home,
            'Weighted PC away previous minute': Weighted_pitch_control_away,
            'Half': half,
            'Time': Time}

    df = pd.DataFrame(data)
    pickle.dump(df, open('pc_over_last_minute' + str(home_files[file][7:14]) + '.p', 'wb'))  
    
        